# Comparison based on Iteration Number

Here, The two algorithms are being compared based on the Number of Iteration required to decode a $n-Bit$ codeword. Bit error rate is calculated by taking the ratio of total number of incorrected bits to toal number of transmitted bits

In [27]:
import numpy as np
from numpy.random import default_rng
from tqdm import tqdm
from matplotlib import pyplot as plt
from pyldpc import make_ldpc, encode, decode, get_message


Bits = np.array([150,200,500,700])
Avg_BER_flip = []

Iteration =200
avg_points = 50
for i in range(len(Bits)):
    
    errors = []
    n = Bits[i]
    d_v = 3
    d_c = 5
    H, G = make_ldpc(n, d_v, d_c, systematic=0, sparse=True)

    ax = n-np.linalg.matrix_rank(H)
    k = G.shape[1]
    kp = np.transpose(G)
    total_error = int(n*(0.10))
    
    for m in range(avg_points):
        msg = np.random.randint(0,2,size = (1,ax))
        cw = msg@kp%2

        error = np.zeros(n)
        
        rng = default_rng()
        random_flip = rng.choice(n, size=total_error, replace=False)
        error[random_flip] = 1

        r = (cw+error)%2

        for iter in range(Iteration):


            wrong_pce = (r@np.transpose(H))%2


            wrong_index = np.where(wrong_pce==1.)[1]


            concat = np.zeros(n)
            nnn = len(wrong_index)

            if(nnn==0):

                break;
            for i in range(nnn):

                concat = np.vstack((concat,H[wrong_index [i],:]))
            concat = np.delete(concat,0,0)

            ones = np.ones(nnn).reshape(nnn)

            flip_mat = ones@concat


            max_ind  = np.where(flip_mat==np.max(flip_mat))
            flip_index = np.random.choice(max_ind[0])



            r[0,flip_index] = (r[0,flip_index]+1)%2

        error_rate = len(np.where(np.equal(r,cw)==False)[0])/n
        errors.append(error_rate)
    Avg_BER_flip.append(np.mean(np.array(errors)))    

In [ ]:
def Initial_prob(c):
    if(c==0):
        return np.log((1-p)/p);

    else:
        return np.log(p/(1-p));

def Mbit(A):
    K = np.ones(len(A))
    check = np.where(M>=0)
    K[check] = 0
    return K 

Avg_BER_BP = []
for i2 in range(len(Bits)):

    n = Bits[i2]

    H, G = make_ldpc(n, d_v, d_c, systematic=0, sparse=True)

    k = G.shape[1]
    kp = np.transpose(G)


    p = 0.1
    ax = n-np.linalg.matrix_rank(H)
    er = int(n*0.1)
    errorsBP = []
    for m in tqdm(range(avg_points)):
        msg = np.random.randint(0,2,size = ax)
        cw = msg@kp%2


        error = np.zeros(n)


        rng = default_rng()
        random_flip = rng.choice(n, size=er, replace=False)
        #print(random_flip)
        error[random_flip] = 1

        r = (cw+error)%2 



        K_0 = np.array([Initial_prob(v) for v in r])


        def g():
            return 1*K_0;

        rows = int((n/d_c)*d_v)
        E_ji = np.zeros((rows,n))
        ier = 0


        for k in range(Iteration):

            if(ier==0):
                M = K_0
         

            for i in range(rows):
                check_eq = H[i,:]
                prev_msg = E_ji[i,:]

                var_node = np.where(check_eq==1)[0]

                M_ij = M[var_node] - prev_msg[var_node]

                tanh_func = np.tanh(M_ij/2)
                product = np.prod(tanh_func)
                for j in range(d_c):

                    E_ji[i,var_node[j]] =  2*np.arctanh(product/tanh_func[j])

            M = K_0+ np.sum(E_ji,axis = 0)


            dw = Mbit(M) 

            if(np.sum(dw@np.transpose(H)%2)==0 ):

                break;
        error_rate_BP = len(np.where(np.equal(dw,cw)==False)[0])/n
        errorsBP.append(error_rate_BP) 
    Avg_BER_BP.append(np.mean(np.array(errorsBP)))    
                


  0%|                                                                                           | 0/50 [00:00<?, ?it/s]<ipython-input-30-3aa7e4213c61>:75: RuntimeWarning: divide by zero encountered in arctanh
  E_ji[i,var_node[j]] =  2*np.arctanh(product/tanh_func[j])
 94%|█████████████████████████████████████████████████████████████████████████████     | 47/50 [03:40<00:15,  5.08s/it]

In [ ]:
plt.figure(figsize =  (15,10))
plt.xlabel("Number of Bits")
plt.ylabel("Bit Error Rate")
plt.title("Error Rate = 10% ,Number of Iteration = "+str(Iteration))
plt.scatter(Bits,Avg_BER_flip,label = "Bit Flip Decoding")
plt.scatter(Bits,Avg_BER_BP,label = "Belief Propagation Decoding")
plt.legend()
plt.show()
